In [1]:
# Import libraries
import os
import pandas as pd

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [2]:
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

### Connect To MIMIC-IV Database

In [3]:
auth.authenticate_user()

In [4]:
# Set up environment variables, Note The Below Information Is Specific To My Google BigQuery Workspace, Please Modify To Your Own Before Running
project_id = 'ml-app-for-health-project'
if project_id == 'name':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Reading data from BigQuery into pandas dataframes.
def run_query(query, project_id = project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id = project_id,
      dialect = 'standard')

# Set the dataset
dataset = 'mimiciv'

### Initial Datasets

In [5]:
# Get all patients with pneumonia (ICD-9 Code: 480 to 486, ICD-10 Code: J12 to J18)
pneumonia_patients = run_query("""
    SELECT DISTINCT
        d.subject_id,
        d.hadm_id AS pneumonia_hadm_id,
        a.admittime AS pneumonia_admittime,
        STRING_AGG(icd_code, ', ') AS pneumonia_icd_codes,
    FROM
        `physionet-data.mimiciv_hosp.diagnoses_icd` d
        INNER JOIN `physionet-data.mimiciv_hosp.admissions` a ON d.hadm_id = a.hadm_id
    WHERE
        (REGEXP_CONTAINS(d.icd_code, '^48[0-6].*') AND d.icd_version = 9) OR
        (REGEXP_CONTAINS(d.icd_code, '^J1[2-8].*') AND d.icd_version = 10)
    GROUP BY
        subject_id, pneumonia_hadm_id, pneumonia_admittime
    ORDER BY
        subject_id, pneumonia_hadm_id, pneumonia_admittime
  """)

pneumonia_patients

,subject_id,pneumonia_hadm_id,pneumonia_admittime,pneumonia_icd_codes
0,10000826,20032235,2146-12-05 19:07:00,486
1,10001176,23334588,2186-11-29 03:56:00,4829
2,10001884,25758848,2128-07-15 18:44:00,486
3,10002155,20345487,2131-03-09 20:33:00,486
4,10002155,23822395,2129-08-04 12:44:00,486
...,...,...,...,...
21841,19998878,29356037,2132-08-17 20:59:00,486
21842,19999287,20175828,2197-08-03 20:58:00,4829
21843,19999287,22997012,2197-07-26 03:29:00,486
21844,19999625,25304202,2139-10-10 18:06:00,486


In [6]:
# Get all patients with heart failure (ICD-9 Code: 428, ICD-10 Code: I50)
heart_failure_patients = run_query("""
    SELECT DISTINCT
        d.subject_id,
        d.hadm_id AS heart_failure_hadm_id,
        a.admittime AS heart_failure_admittime,
        STRING_AGG(icd_code, ', ') AS heart_failure_icd_codes,
    FROM
        `physionet-data.mimiciv_hosp.diagnoses_icd` d
        INNER JOIN `physionet-data.mimiciv_hosp.admissions` a ON d.hadm_id = a.hadm_id
    WHERE
        (d.icd_code LIKE "428%" AND d.icd_version = 9) OR
        (d.icd_code LIKE "I50%" AND d.icd_version = 10)
    GROUP BY
        subject_id, heart_failure_hadm_id, heart_failure_admittime
    ORDER BY
        subject_id, heart_failure_hadm_id, heart_failure_admittime
  """)

heart_failure_patients

,subject_id,heart_failure_hadm_id,heart_failure_admittime,heart_failure_icd_codes
0,10000980,20897796,2193-08-15 01:01:00,I5033
1,10000980,24947999,2190-11-06 20:57:00,"42823, 4280"
2,10000980,25242409,2191-04-03 18:48:00,"42832, 4280"
3,10000980,25911675,2191-05-23 15:33:00,I5023
4,10000980,26913865,2189-06-27 07:38:00,"42823, 4280"
...,...,...,...,...
60936,19998330,24492004,2178-10-01 07:28:00,"42833, 4280"
60937,19998330,27282608,2177-07-25 04:34:00,"42832, 4280"
60938,19998497,24819607,2139-09-09 13:01:00,4280
60939,19998591,23299044,2185-03-10 14:19:00,4280


In [7]:
# Join pneumonia and heart failure dataframes to see which pneumonia patients got heart failure within next 6 months
six_months = pd.DateOffset(months = 6)
patients_df = pneumonia_patients.copy()
patients_df["has_heart_failure"] = False

for p_index, p_row in patients_df.iterrows():
    subject_id = p_row["subject_id"]
    pneumonia_admittime = pd.Timestamp(p_row["pneumonia_admittime"])
    same_subject_hf_entries = heart_failure_patients[heart_failure_patients["subject_id"] == subject_id]

    for hf_index, hf_row in same_subject_hf_entries.iterrows():
        heart_failure_admittime = pd.Timestamp(hf_row["heart_failure_admittime"])
        time_diff = heart_failure_admittime - pneumonia_admittime

        if (heart_failure_admittime > pneumonia_admittime and heart_failure_admittime <= pneumonia_admittime + six_months):
            for col in heart_failure_patients.columns:
                if col != "subject_id":
                    patients_df.at[p_index, col] = hf_row[col]
                    patients_df[col] = patients_df[col].astype(heart_failure_patients[col].dtype)
            patients_df.at[p_index, "has_heart_failure"] = True
            break

patients_df

,subject_id,pneumonia_hadm_id,pneumonia_admittime,pneumonia_icd_codes,has_heart_failure,heart_failure_hadm_id,heart_failure_admittime,heart_failure_icd_codes
0,10000826,20032235,2146-12-05 19:07:00,486,False,<NA>,NaT,NaN
1,10001176,23334588,2186-11-29 03:56:00,4829,False,<NA>,NaT,NaN
2,10001884,25758848,2128-07-15 18:44:00,486,False,<NA>,NaT,NaN
3,10002155,20345487,2131-03-09 20:33:00,486,False,<NA>,NaT,NaN
4,10002155,23822395,2129-08-04 12:44:00,486,False,<NA>,NaT,NaN
...,...,...,...,...,...,...,...,...
21841,19998878,29356037,2132-08-17 20:59:00,486,False,<NA>,NaT,NaN
21842,19999287,20175828,2197-08-03 20:58:00,4829,False,<NA>,NaT,NaN
21843,19999287,22997012,2197-07-26 03:29:00,486,False,<NA>,NaT,NaN
21844,19999625,25304202,2139-10-10 18:06:00,486,False,<NA>,NaT,NaN


In [8]:
patients_df["has_heart_failure"].value_counts()

,count
has_heart_failure,
False,18749
True,3097


### Filtering Based On hadm_ids

In [9]:
subject_ids = patients_df["subject_id"].unique()
subject_ids_str = ", ".join(map(str, subject_ids))

relevant_hadm_ids = run_query(f"""
    SELECT DISTINCT
        subject_id, hadm_id, admittime
    FROM
        `physionet-data.mimiciv_hosp.admissions`
    WHERE
        subject_id IN ({subject_ids_str})
    ORDER BY
        subject_id, admittime
""")

relevant_hadm_ids

,subject_id,hadm_id,admittime
0,10000826,20032235,2146-12-05 19:07:00
1,10000826,21086876,2146-12-18 17:39:00
2,10000826,28289260,2146-12-31 00:43:00
3,10001176,23334588,2186-11-29 03:56:00
4,10001884,21268656,2125-10-18 16:50:00
...,...,...,...
86752,19999287,25875727,2191-12-29 07:15:00
86753,19999287,22997012,2197-07-26 03:29:00
86754,19999287,20175828,2197-08-03 20:58:00
86755,19999625,27638769,2138-10-06 17:27:00


In [10]:
def get_filtered_hadm_ids(subject_id, time):
    six_months = pd.DateOffset(months = 6)
    end_time = time + six_months
    # Only between the admittime of pneumonia diagnosis and the next 6 months
    only_within_6_months = relevant_hadm_ids[(relevant_hadm_ids["subject_id"] == subject_id) & (relevant_hadm_ids["admittime"] > time) & (relevant_hadm_ids["admittime"] <= end_time)]
    only_within_6_months_hadm_ids = ", ".join(only_within_6_months["hadm_id"].astype(str))
    # Everything till the 6 months date from pneumonia diagnosis (before penumonia + pneumonia diagnosis + 6 months after that)
    everything_till_6_months_end = relevant_hadm_ids[(relevant_hadm_ids["subject_id"] == subject_id) & (relevant_hadm_ids["admittime"] <= end_time)]
    everything_till_6_months_end_hadm_ids = ", ".join(everything_till_6_months_end["hadm_id"].astype(str))

    return only_within_6_months_hadm_ids, everything_till_6_months_end_hadm_ids

patients_df["hadm_ids_within_6_months"], patients_df["hadm_ids_everything_till_6_months"] = zip(*patients_df.apply(lambda row: get_filtered_hadm_ids(row["subject_id"], row["pneumonia_admittime"]), axis = 1))

In [11]:
# Do some refactoring of non-null values
patients_df.replace("", pd.NA, inplace = True)
patients_df.replace("nan", pd.NA, inplace = True)
patients_df

,subject_id,pneumonia_hadm_id,pneumonia_admittime,pneumonia_icd_codes,has_heart_failure,heart_failure_hadm_id,heart_failure_admittime,heart_failure_icd_codes,hadm_ids_within_6_months,hadm_ids_everything_till_6_months
0,10000826,20032235,2146-12-05 19:07:00,486,False,<NA>,NaT,NaN,"21086876, 28289260","20032235, 21086876, 28289260"
1,10001176,23334588,2186-11-29 03:56:00,4829,False,<NA>,NaT,NaN,<NA>,23334588
2,10001884,25758848,2128-07-15 18:44:00,486,False,<NA>,NaT,NaN,<NA>,"21268656, 26679629, 23594368, 21577720, 243258..."
3,10002155,20345487,2131-03-09 20:33:00,486,False,<NA>,NaT,NaN,<NA>,"28439444, 23822395, 28976727, 28994087, 20345487"
4,10002155,23822395,2129-08-04 12:44:00,486,False,<NA>,NaT,NaN,28976727,"28439444, 23822395, 28976727"
...,...,...,...,...,...,...,...,...,...,...
21841,19998878,29356037,2132-08-17 20:59:00,486,False,<NA>,NaT,NaN,"26489544, 21643535","27724939, 27930413, 27554506, 29356037, 264895..."
21842,19999287,20175828,2197-08-03 20:58:00,4829,False,<NA>,NaT,NaN,<NA>,"25875727, 22997012, 20175828"
21843,19999287,22997012,2197-07-26 03:29:00,486,False,<NA>,NaT,NaN,20175828,"25875727, 22997012, 20175828"
21844,19999625,25304202,2139-10-10 18:06:00,486,False,<NA>,NaT,NaN,<NA>,"27638769, 25304202"


In [12]:
# Exclude pneumonia patients who have no hospital admissions in the next 6 months
patients_df_filtered = patients_df[patients_df["hadm_ids_within_6_months"].notnull()].copy()
patients_df_filtered = patients_df_filtered.reset_index(drop = True)
patients_df_filtered["has_heart_failure"].value_counts()

,count
has_heart_failure,
False,5580
True,3097


In [13]:
# Some hadm_id related variables which will help in queries going forward
hadm_ids = patients_df_filtered["hadm_ids_everything_till_6_months"]
hadm_ids_str = ", ".join(map(str, hadm_ids))
patients_df_filtered["hadm_ids_everything_list"] = patients_df_filtered["hadm_ids_everything_till_6_months"].apply(lambda x: x.split(", "))

### Comorbidities

In [14]:
# List of comorbidities and their related ICD-9 and ICD-10 codes
comorbidities_dict = {
    "Hypertension": ["401", "I10"],
    "Heartburn": ["7871", "R12"],
    "Heart valve replaced by transplant": ["V422", "Z953"],
    "Heart valve replaced by other means": ["V432", "Z952"],
    "Chronic kidney disease": ["585", "N18"],
    "Obesity": ["278", "E66"],
    "Hyperlipidemia": ["272", "E78"],
}

In [15]:
# Find the patients (based on subject_id and hadm_id) who have the above comorbidities
for comorbidity, icd_codes in comorbidities_dict.items():
    like_conditions = " OR ".join([f"icd_code LIKE '{icd_code}%'" for icd_code in icd_codes])
    comorbidity_query = run_query(f"""
        SELECT DISTINCT
            subject_id, hadm_id,
        FROM
            `physionet-data.mimiciv_hosp.diagnoses_icd`
        WHERE
            hadm_id IN ({hadm_ids_str}) AND
            ({like_conditions})
    """)

    comorbidity_hadm_ids = set(comorbidity_query["hadm_id"])
    patients_df_filtered[comorbidity] = patients_df_filtered["hadm_ids_everything_list"].apply(lambda x: int(any(int(i) in comorbidity_hadm_ids for i in x)))
    print(f"""{comorbidity} (by subject_id): {patients_df_filtered.drop_duplicates(subset = "subject_id")[comorbidity].sum()} / {patients_df_filtered["subject_id"].nunique()}""")
    print(f"""{comorbidity} (by hadm_id): {patients_df_filtered[comorbidity].sum()} / {len(patients_df_filtered)}\n""")

Hypertension (by subject_id): 3670 / 6173
Hypertension (by hadm_id): 5317 / 8677

Heartburn (by subject_id): 30 / 6173
Heartburn (by hadm_id): 50 / 8677

Heart valve replaced by transplant (by subject_id): 146 / 6173
Heart valve replaced by transplant (by hadm_id): 189 / 8677

Heart valve replaced by other means (by subject_id): 96 / 6173
Heart valve replaced by other means (by hadm_id): 141 / 8677

Chronic kidney disease (by subject_id): 2285 / 6173
Chronic kidney disease (by hadm_id): 3403 / 8677

Obesity (by subject_id): 1095 / 6173
Obesity (by hadm_id): 1595 / 8677

Hyperlipidemia (by subject_id): 3315 / 6173
Hyperlipidemia (by hadm_id): 4745 / 8677



### Medications

In [16]:
# List of medication names
medications_dict = {
    "ACE Inhibitors": ["enalapril", "lisinopril"],
    "Beta Blockers": ["metoprolol", "carvedilol", "bisoprolol"],
    "Diuretics": ["furosemide", "spironolactone"],
    "Angiotensin II Receptor Blockers": ["losartan", "valsartan"],
    "Aldosterone Antagonists": ["spironolactone", "eplerenone"],
    "Digoxin": ["digoxin"],
    "Vasodilators": ["hydralazine", "isosorbide dinitrate"],
}

In [17]:
# Find the patients (based on subject_id and hadm_id) who have been administered the above medications
for medication_group, medication_names in medications_dict.items():
    like_conditions = " OR ".join([f"LOWER(medication) LIKE '{medication_name}%'" for medication_name in medication_names])
    medication_query = run_query(f"""
        SELECT DISTINCT
            subject_id, hadm_id,
        FROM
            `physionet-data.mimiciv_hosp.emar`
        WHERE
            hadm_id IN ({hadm_ids_str}) AND
            event_txt LIKE '%Administered%' AND
            ({like_conditions})
    """)

    medication_hadm_ids = set(medication_query["hadm_id"])
    patients_df_filtered[medication_group] = patients_df_filtered["hadm_ids_everything_list"].apply(lambda x: int(any(int(i) in medication_hadm_ids for i in x)))
    print(f"""{medication_group} (by subject_id): {patients_df_filtered.drop_duplicates(subset = "subject_id")[medication_group].sum()} / {patients_df_filtered["subject_id"].nunique()}""")
    print(f"""{medication_group} (by hadm_id): {patients_df_filtered[medication_group].sum()} / {len(patients_df_filtered)}\n""")

ACE Inhibitors (by subject_id): 766 / 6173
ACE Inhibitors (by hadm_id): 1043 / 8677

Beta Blockers (by subject_id): 1639 / 6173
Beta Blockers (by hadm_id): 2312 / 8677

Diuretics (by subject_id): 1863 / 6173
Diuretics (by hadm_id): 2619 / 8677

Angiotensin II Receptor Blockers (by subject_id): 414 / 6173
Angiotensin II Receptor Blockers (by hadm_id): 545 / 8677

Aldosterone Antagonists (by subject_id): 360 / 6173
Aldosterone Antagonists (by hadm_id): 485 / 8677

Digoxin (by subject_id): 160 / 6173
Digoxin (by hadm_id): 224 / 8677

Vasodilators (by subject_id): 672 / 6173
Vasodilators (by hadm_id): 936 / 8677



### Lab Events

In [18]:
# Function to get all relevant lab events in a single query
def search_all_labevents(lab_ids, hadm_ids):
    lab_ids_str = ", ".join([str(lab_id) for lab_id in lab_ids])  # Convert list of lab_ids to comma-separated string
    hadm_ids_str = ", ".join([str(hadm_id) for hadm_id in hadm_ids])  # Convert list of hadm_ids to string
    query = f"""
        SELECT
            subject_id, hadm_id, itemid AS labevent_id, valuenum
        FROM
            `physionet-data.mimiciv_hosp.labevents`
        WHERE
            hadm_id IN ({hadm_ids_str}) AND
            itemid IN ({lab_ids_str})
    """
    return run_query(query)

# List of lab IDs
lab_ids = [50963, 5192, 51002, 52024, 50912, 52546, 51977, 51082, 50883, 50884, 50885, 50878, 50861]

# Extract the necessary fields from the original patients_df_filtered
df_subset = patients_df_filtered[["subject_id", "pneumonia_hadm_id", "hadm_ids_within_6_months"]].copy()
df_subset["hadm_ids_list"] = df_subset["hadm_ids_within_6_months"].str.split(", ").apply(lambda x: [int(i) for i in x])
df_subset["hadm_ids_list"] = df_subset.apply(lambda row: row["hadm_ids_list"] + [int(row["pneumonia_hadm_id"])], axis = 1)

# Expand the list to separate lines
df_exploded = df_subset.explode("hadm_ids_list")

# Get unique HADM IDs
unique_hadm_ids = df_exploded["hadm_ids_list"].unique()

# Run query
labevents_df = search_all_labevents(lab_ids, unique_hadm_ids)
labevents_df = pd.DataFrame(labevents_df)

# Calculate min, max, and avg for each lab test for each subject/hadm_id, ignoring NaN values
labevents_summary_df = labevents_df.groupby(["hadm_id", "labevent_id"]).agg(
    min = ("valuenum", lambda x: x.min(skipna = True)),
    max = ("valuenum", lambda x: x.max(skipna = True)),
    avg = ("valuenum", lambda x: x.mean(skipna = True))
).reset_index()

# Pivot the table to have separate columns for each lab_id's min, max, and avg
labevents_pivot_df = labevents_summary_df.pivot_table(index = ["hadm_id"], columns = "labevent_id", values = ["min", "max", "avg"])

# Flatten the column names dynamically based on actual pivoted columns, and remove "_val"
labevents_pivot_df.columns = [f"{lab_id}_{agg}" for agg, lab_id in labevents_pivot_df.columns]

# Sort columns to have min, max, avg order for each lab_id
sorted_columns = sorted(labevents_pivot_df.columns, key = lambda x: (x.split("_")[0], ["min", "max", "avg"].index(x.split("_")[1])))
labevents_pivot_df = labevents_pivot_df[sorted_columns]
labevents_pivot_df.reset_index(inplace = True)

In [19]:
# Merge statistics back into df_exploded
labevents_pivot_df["hadm_id"] = labevents_pivot_df["hadm_id"].astype(int)

labevents_merged_df = df_exploded.merge(labevents_pivot_df, left_on = "hadm_ids_list", right_on = "hadm_id", how = "left")

# Aggregate as needed, keep all min, max, avg columns
stat_columns = [col for col in labevents_merged_df.columns if any(suffix in col for suffix in ["_min", "_max", "_avg"])]

# Categorize the statistic columns into min, max, avg
min_cols = [col for col in stat_columns if "_min" in col]
max_cols = [col for col in stat_columns if "_max" in col]
avg_cols = [col for col in stat_columns if "_avg" in col]

# Define the aggregation dictionary
agg_dict = {}
for col in min_cols:
    agg_dict[col] = "min"
for col in max_cols:
    agg_dict[col] = "max"
for col in avg_cols:
    agg_dict[col] = "mean"

# Perform aggregation
labevents_final_output = labevents_merged_df.groupby(["subject_id", "pneumonia_hadm_id", "hadm_ids_within_6_months"])[stat_columns].agg(agg_dict).reset_index()
labevents_final_output = labevents_final_output.sort_values(by = ["subject_id", "pneumonia_hadm_id"])
labevents_final_output.columns = [col.replace("_val", "") if "_val" in col else col for col in labevents_final_output.columns]

stat_columns_final = [col for col in labevents_final_output.columns if any(suffix in col for suffix in ["_min", "_max", "_avg"])]

# Extract all lab_ids
lab_ids_final = sorted(set(col.split("_")[0] for col in stat_columns_final))

# Create an ordered list of statistical columns
ordered_stat_cols_final = []
for lab_id in lab_ids_final:
    for stat in ["min", "max", "avg"]:
        col_name = f"{lab_id}_{stat}"
        if col_name in labevents_final_output.columns:
            ordered_stat_cols_final.append(col_name)

# Define new column order
new_order_final = ["subject_id", "pneumonia_hadm_id", "hadm_ids_within_6_months"] + ordered_stat_cols_final
labevents_final_output = labevents_final_output[new_order_final]
labevents_final_output.reset_index(drop = True, inplace = True)

In [20]:
final_output = pd.merge(patients_df_filtered, labevents_final_output, how = "left", on = ["subject_id", "pneumonia_hadm_id", "hadm_ids_within_6_months"])

### Chart Events

In [21]:
item_ids = run_query("""
    SELECT *
    FROM
        `physionet-data.mimiciv_icu.d_items`
    WHERE
        label LIKE "%Heart Rate%" OR
        label LIKE "%Oxygen saturation%" OR
        label LIKE "%O2 saturation pulseoxymetry%" OR
        label LIKE "%Respiratory Rate%" OR
        label LIKE "%Central Venous Pressure%" OR
        label LIKE "%Temperature%" OR
        label LIKE "%Arterial Blood Pressure%"
""")
item_ids

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,224027,Skin Temperature,Skin Temp,chartevents,Skin - Assessment,None,Text,NaN,NaN
1,228242,Pt. Temperature (BG) (SOFT),Pt. Temperature (BG) (SOFT),chartevents,Labs,None,Numeric,NaN,NaN
2,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
3,220056,Arterial Blood Pressure Alarm - Low,ABP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN
4,220058,Arterial Blood Pressure Alarm - High,ABP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
5,220072,Central Venous Pressure Alarm - High,CVP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
6,220073,Central Venous Pressure Alarm - Low,CVP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN
7,224674,Changes in Temperature,Changes in Temperature,chartevents,Toxicology,None,Text,NaN,NaN
8,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
9,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN


In [22]:
itemid_dict = {"223761": "Temperature Fahrenheit",
               "220210": "Respiratory Rate",
               "220074": "Central Venous Pressure",
               "228232": "Oxygen Saturation",
               "220277": "O2 Saturation Pulseoxymetry",
               "220045": "Heart Rate",
               "220050": "Arterial Blood Pressure systolic",
               "220051": "Arterial Blood Pressure diastolic",
               "220052": "Arterial Blood Pressure mean"}

In [23]:
# Function to get all relevant chart events in a single query
def search_all_chartevents(chart_ids, hadm_ids):
    chart_ids_str = ", ".join([str(chart_id) for chart_id in chart_ids])  # Convert list of chart_ids to comma-separated string
    hadm_ids_str = ", ".join([str(hadm_id) for hadm_id in hadm_ids])  # Convert list of hadm_ids to string
    query = f"""
        SELECT
            subject_id, hadm_id, itemid AS chartevent_id, valuenum
        FROM
            `physionet-data.mimiciv_icu.chartevents`
        WHERE
            hadm_id IN ({hadm_ids_str}) AND
            itemid IN ({chart_ids_str})
    """
    return run_query(query)

# List of chart IDs
chart_ids = list(itemid_dict.keys())

# Extract the necessary fields from the original patients_df_filtered
df_subset = patients_df_filtered[["subject_id", "pneumonia_hadm_id", "hadm_ids_within_6_months"]].copy()
df_subset["hadm_ids_list"] = df_subset["hadm_ids_within_6_months"].str.split(", ").apply(lambda x: [int(i) for i in x])
df_subset["hadm_ids_list"] = df_subset.apply(lambda row: row["hadm_ids_list"] + [int(row["pneumonia_hadm_id"])], axis = 1)

# Expand the list to separate lines
df_exploded = df_subset.explode("hadm_ids_list")

# Get unique HADM IDs
unique_hadm_ids = df_exploded["hadm_ids_list"].unique()

# Run query
chartevents_df = search_all_chartevents(chart_ids, unique_hadm_ids)
chartevents_df = pd.DataFrame(chartevents_df)

# Calculate min, max, and avg for each chart event for each subject/hadm_id, ignoring NaN values
chartevents_summary_df = chartevents_df.groupby(["hadm_id", "chartevent_id"]).agg(
    min = ("valuenum", lambda x: x.min(skipna = True)),
    max = ("valuenum", lambda x: x.max(skipna = True)),
    avg = ("valuenum", lambda x: x.mean(skipna = True))
).reset_index()

# Pivot the table to have separate columns for each chart_id's min, max, and avg
chartevents_pivot_df = chartevents_summary_df.pivot_table(index = ["hadm_id"], columns = "chartevent_id", values = ["min", "max", "avg"])

# Flatten the column names dynamically based on actual pivoted columns, and remove "_val"
chartevents_pivot_df.columns = [f"{chart_id}_{agg}" for agg, chart_id in chartevents_pivot_df.columns]

# Sort columns to have min, max, avg order for each chart_id
sorted_columns = sorted(chartevents_pivot_df.columns, key = lambda x: (x.split("_")[0], ["min", "max", "avg"].index(x.split("_")[1])))
chartevents_pivot_df = chartevents_pivot_df[sorted_columns]
chartevents_pivot_df.reset_index(inplace = True)

In [24]:
# Merge statistics back into df_exploded
chartevents_pivot_df["hadm_id"] = chartevents_pivot_df["hadm_id"].astype(int)

chartevents_merged_df = df_exploded.merge(chartevents_pivot_df, left_on = "hadm_ids_list", right_on = "hadm_id", how = "left")

# Aggregate as needed, keep all min, max, avg columns
stat_columns = [col for col in chartevents_merged_df.columns if any(suffix in col for suffix in ["_min", "_max", "_avg"])]

# Categorize the statistic columns into min, max, avg
min_cols = [col for col in stat_columns if "_min" in col]
max_cols = [col for col in stat_columns if "_max" in col]
avg_cols = [col for col in stat_columns if "_avg" in col]

# Define the aggregation dictionary
agg_dict = {}
for col in min_cols:
    agg_dict[col] = "min"
for col in max_cols:
    agg_dict[col] = "max"
for col in avg_cols:
    agg_dict[col] = "mean"

# Perform aggregation
chartevents_final_output = chartevents_merged_df.groupby(["subject_id", "pneumonia_hadm_id", "hadm_ids_within_6_months"])[stat_columns].agg(agg_dict).reset_index()
chartevents_final_output = chartevents_final_output.sort_values(by = ["subject_id", "pneumonia_hadm_id"])
chartevents_final_output.columns = [col.replace("_val", "") if "_val" in col else col for col in chartevents_final_output.columns]

stat_columns_final = [col for col in chartevents_final_output.columns if any(suffix in col for suffix in ["_min", "_max", "_avg"])]

# Extract all chart_ids
chart_ids_final = sorted(set(col.split("_")[0] for col in stat_columns_final))

# Create an ordered list of statistical columns
ordered_stat_cols_final = []
for chart_id in chart_ids_final:
    for stat in ["min", "max", "avg"]:
        col_name = f"{chart_id}_{stat}"
        if col_name in chartevents_final_output.columns:
            new_col_name = f"{itemid_dict[chart_id]}_{stat}"
            chartevents_final_output.rename(columns = {col_name: new_col_name}, inplace = True)
            ordered_stat_cols_final.append(new_col_name)

# Define new column order
new_order_final = ["subject_id", "pneumonia_hadm_id", "hadm_ids_within_6_months"] + ordered_stat_cols_final
chartevents_final_output = chartevents_final_output[new_order_final]
chartevents_final_output.reset_index(drop = True, inplace = True)

In [25]:
final_output = pd.merge(final_output, chartevents_final_output, how = "left", on = ["subject_id", "pneumonia_hadm_id", "hadm_ids_within_6_months"])

### Additional Patient Details

In [26]:
subject_ids = patients_df_filtered["subject_id"]
subject_ids_str = ", ".join(map(str, subject_ids))

pneumonia_hadm_ids = patients_df_filtered["pneumonia_hadm_id"]
pneumonia_hadm_ids_str = ", ".join(map(str, pneumonia_hadm_ids))

In [27]:
# Get the gender of each patient
patients_gender = run_query(f"""
    SELECT DISTINCT
        subject_id, gender,
    FROM
        `physionet-data.mimiciv_hosp.patients`
    WHERE
        subject_id IN ({subject_ids_str})
""")
patients_gender["gender"] = patients_gender["gender"].map({"M": 1, "F": 0})

final_output = pd.merge(final_output, patients_gender, how = "left", on = ["subject_id"])

In [28]:
# Get the age of each patient, based on the date of hospital admission
patients_age = run_query(f"""
    SELECT DISTINCT
        subject_id, hadm_id AS pneumonia_hadm_id, age
    FROM
        `physionet-data.mimiciv_derived.age`
    WHERE
        hadm_id IN ({pneumonia_hadm_ids_str})
""")

final_output = pd.merge(final_output, patients_age, how = "left", on = ["subject_id", "pneumonia_hadm_id"])

### Output To CSV File

In [29]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
final_output.head()

,subject_id,pneumonia_hadm_id,pneumonia_admittime,pneumonia_icd_codes,has_heart_failure,heart_failure_hadm_id,heart_failure_admittime,heart_failure_icd_codes,hadm_ids_within_6_months,hadm_ids_everything_till_6_months,hadm_ids_everything_list,Hypertension,Heartburn,Heart valve replaced by transplant,Heart valve replaced by other means,Chronic kidney disease,Obesity,Hyperlipidemia,ACE Inhibitors,Beta Blockers,Diuretics,Angiotensin II Receptor Blockers,Aldosterone Antagonists,Digoxin,Vasodilators,50861_min,50861_max,50861_avg,50878_min,50878_max,50878_avg,50883_min,50883_max,50883_avg,50884_min,50884_max,50884_avg,50885_min,50885_max,50885_avg,50912_min,50912_max,50912_avg,50963_min,50963_max,50963_avg,51082_min,51082_max,51082_avg,52024_min,52024_max,52024_avg,Heart Rate_min,Heart Rate_max,Heart Rate_avg,Arterial Blood Pressure systolic_min,Arterial Blood Pressure systolic_max,Arterial Blood Pressure systolic_avg,Arterial Blood Pressure diastolic_min,Arterial Blood Pressure diastolic_max,Arterial Blood Pressure diastolic_avg,Arterial Blood Pressure mean_min,Arterial Blood Pressure mean_max,Arterial Blood Pressure mean_avg,Central Venous Pressure_min,Central Venous Pressure_max,Central Venous Pressure_avg,Respiratory Rate_min,Respiratory Rate_max,Respiratory Rate_avg,O2 Saturation Pulseoxymetry_min,O2 Saturation Pulseoxymetry_max,O2 Saturation Pulseoxymetry_avg,Temperature Fahrenheit_min,Temperature Fahrenheit_max,Temperature Fahrenheit_avg,Oxygen Saturation_min,Oxygen Saturation_max,Oxygen Saturation_avg,gender,age
0,10000826,20032235,2146-12-05 19:07:00,486,False,<NA>,NaT,NaN,"21086876, 28289260","20032235, 21086876, 28289260","[20032235, 21086876, 28289260]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,27.0,56.0,37.420635,100.0,335.0,147.968254,NaN,NaN,NaN,NaN,NaN,NaN,1.3,2.7,1.942857,0.3,0.5,0.394048,NaN,NaN,NaN,94.0,190.0,131.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,32
1,10002155,23822395,2129-08-04 12:44:00,486,False,<NA>,NaT,NaN,28976727,"28439444, 23822395, 28976727","[28439444, 23822395, 28976727]",1,0,0,0,1,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,2.5,1.657292,NaN,NaN,NaN,45.0,46.0,45.50,NaN,NaN,NaN,61.0,108.0,82.883871,98.0,124.0,113.400000,46.0,64.0,57.400000,67.0,88.0,80.200000,NaN,NaN,NaN,10.0,22.0,16.292208,81.0,99.0,92.911950,95.9,98.8,97.430556,NaN,NaN,NaN,0,81
2,10002155,28994087,2130-09-23 21:59:00,486,False,<NA>,NaT,NaN,20345487,"28439444, 23822395, 28976727, 28994087, 20345487","[28439444, 23822395, 28976727, 28994087, 20345...",1,0,0,0,1,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4,2.5,1.883333,NaN,NaN,NaN,65.0,65.0,65.00,NaN,NaN,NaN,68.0,142.0,101.789583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,37.0,18.737981,80.0,100.0,94.860234,94.0,99.4,96.840909,NaN,NaN,NaN,0,82
3,10002428,28662225,2156-04-12 14:16:00,486,True,23473524,2156-05-11 14:49:00,"42843, 4280","20321825, 23473524","25797028, 28662225, 20321825, 23473524","[25797028, 28662225, 20321825, 23473524]",1,0,0,0,0,0,1,0,0,0,0,0,0,0,44.0,66.0,53.500000,52.0,102.0,80.333333,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.2,0.125000,0.3,1.3,0.464646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,135.0,96.404167,73.0,176.0,112.150305,35.0,156.0,57.872482,53.0,208.0,79.415225,4.0,310.0,50.243590,11.0,34.0,21.280153,73.0,100.0,98.323080,97.0,102.9,98.588215,NaN,NaN,NaN,0,81
4,10003019,22774359,2175-10-08 13:56:00,486,False,<NA>,NaT,NaN,"21223482, 20277210, 21213148, 25179393, 20962108","23693618, 25573783, 21616816, 24646702, 200301...","[23693618, 25573783, 21616816, 24646702, 20030...",0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.0,43.0,34.257143,13.0,81.0,34.742857,NaN,NaN,NaN,NaN,NaN,NaN,0.2,0.6,0.347143,0.7,1.3,0.952727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,105.0,81.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,13.0,10.846154,8.0,23.0,17.833333,95.0,99.0,96.888889,97.6,98.3,97.875000,NaN,NaN,NaN,1,73


In [30]:
final_output.to_csv("final_patients_dataset.csv", index = False)